# 01 - Recordfile Preprocessing

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip -q
!{sys.executable} -m pip install pandas -q
!{sys.executable}  -m pip install numpy -q
!{sys.executable}  -m pip install matplotlib -q
!{sys.executable} -m pip install mgz -q

In [2]:
cd ..

/home/niel


In [3]:
import numpy as np
import pandas as pd
import json
import os
from mgz import header, body
from mgz.model import parse_match
import pickle
import os
import zipfile

In [4]:
#settings
from_pkl = False
from_directory = True

base_directory = "aoe_data/data/scraped_matches/"
recordfile_path = base_directory +"unzipped_recordfiles/"


In [5]:
if from_pkl:
    #load match list from pkl
    with open('aoe_data/data/aoe2insights_20k_matchlist.pkl', 'rb') as f:
        match_list = pickle.load(f)

    match_list = match_list.tolist()

In [6]:
if from_directory:
    
    match_list = []
    #unzip all files
    directory = os.fsencode(recordfile_path)

    files = len(os.listdir(directory))
    print("Found ",files, " files.")
    
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        match_id = filename[15:-11]
        match_list.append(match_id)
        
print(len(match_list), " Matchfiles found")

Found  12349  files.
12349  Matchfiles found


**Data Preparation**

In [7]:
import os
#prepare files
directory = os.fsencode(recordfile_path)
filenames = []

#get all file names
for file in os.listdir(directory):
  filename = os.fsdecode(file)
  filenames.append(filename)

print(len(filenames))
print(filenames[1])

12349
AgeIIDE_Replay_155650737.aoe2record


In [8]:
matches = {}
error_count = 0

for match_id in match_list:
  if error_count >= 1000:
    break

  filename = 'AgeIIDE_Replay_{}.aoe2record'.format(match_id)

  if filename in filenames:
    #create dict for each match
    matches[match_id] = {}
    matches[match_id]['match_id'] = match_id
    matches[match_id]['recordfile'] = filename

    

In [ ]:
#read in data from record files

##directorys
recordfile_path = recordfile_path
input_path = 'aoe_data/data/scraped_matches/inputs/'
gaia_path = 'aoe_data/data/scraped_matches/gaia_data/'

## orga vars
error_count = 0
parsed_files = 0
to_be_deleted = []
input_dict = {}
gaia_dict = {}

## content vars for feature creation
masterdata_dict = {}
masterdata_dict['Build'] = []
masterdata_dict['Queue'] = []
masterdata_dict['Research'] = []

##set to True if recreation of record files needed
parse_recordfiles = True

for match_index,match_id in enumerate(matches):
    try:
        filepath = recordfile_path + matches[match_id]['recordfile']
        matches[match_id]['input_fn'] = input_path + str(match_id) + ".pkl"
        
        if parse_recordfiles:

            #parse match data
            with open(filepath, 'rb') as data:
                eof = os.fstat(data.fileno()).st_size
                body.meta.parse_stream(data)
                while data.tell() < eof:
                    body.operation.parse_stream(data)

            with open(filepath, 'rb') as data:
                match = parse_match(data)
                
            #get general game info  
            matches[match_id]['map'] = match.map.name
            matches[match_id]['map_size'] = match.map.size
            matches[match_id]['duration'] = match.duration.seconds

            #gather player information
            players = match.players
            matches[match_id]['p1_name'] = players[0].name
            matches[match_id]['p2_name'] = players[1].name

            matches[match_id]['p1_civ'] = players[0].civilization
            matches[match_id]['p2_civ'] = players[1].civilization

            matches[match_id]['p1_xpos'] = players[0].position.x
            matches[match_id]['p2_xpos'] = players[1].position.x
            matches[match_id]['p1_ypos'] = players[0].position.y
            matches[match_id]['p2_ypos'] = players[1].position.y


            if players[0].winner == True:
                matches[match_id]['winner'] = int(0)
            else:
                matches[match_id]['winner'] = int(1)


            #get all item names
            for object in match.inputs:
                attr_list = object.__dict__.keys()
                break

            if not str(getattr(object,'type')) == "Chat":
                inputs_objects_dict = {}

            for index, object in enumerate(match.inputs):
                inputs_objects_dict[index] = {}

                #create timestamp_seconds
                inputs_objects_dict[index]['ts_seconds'] = getattr(object,'timestamp').seconds

                for attribute in attr_list:
                    inputs_objects_dict[index][attribute] = str(getattr(object,attribute))


                    #create a list of Builds
                    if str(getattr(object,'type')) == 'Build':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Build'] :
                            masterdata_dict['Build'] .append(val)

                    #create list of possible Units
                    if str(getattr(object,'type')) == 'Queue':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Queue'] :
                            masterdata_dict['Queue'] .append(val)

                    #create list of Technologies
                    if str(getattr(object,'type')) == 'Research':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Research'] :
                            masterdata_dict['Research'].append(val)


                    #create redundant single xy coordinates
                    if attribute == 'position':
                        pos = getattr(object,attribute)

                        if pos == None:
                            inputs_objects_dict[index]['x_pos']= None
                            inputs_objects_dict[index]['y_pos'] = None 
                        else:
                            inputs_objects_dict[index]["x_pos"] = getattr(object,attribute).x
                            inputs_objects_dict[index]["y_pos"] = getattr(object,attribute).y

                    #replace playerinfo
                    if attribute == 'player':
                        if inputs_objects_dict[index][attribute] == matches[match_id]['p1_name']:
                            inputs_objects_dict[index][attribute] = 'p1'
                        else:
                            inputs_objects_dict[index][attribute] = 'p2'


            #read gaia data
            gaia_objects_dict = {}
            for object in match.gaia:
                if getattr(object,"name"):
                    #read gaia data in
                    gaia_objects_dict[object.instance_id] = {}
                    gaia_objects_dict[object.instance_id]["name"] = getattr(object,"name")
                    gaia_objects_dict[object.instance_id]["x_pos"] = getattr(object,'position').x
                    gaia_objects_dict[object.instance_id]["y_pos"] = getattr(object,'position').y

            #write dicts into global dict
            input_dict[match_id] = inputs_objects_dict
            gaia_dict[match_id] = gaia_objects_dict

            #persistate inputs in pkl file
            matches[match_id]['input_fn'] = input_path + str(match_id) + ".pkl"
            output = open(matches[match_id]['input_fn'] , 'wb')
            pickle.dump(inputs_objects_dict, output)
            output.close()

            #persitate gaia_data
            matches[match_id]['gaia_fn'] = gaia_path + str(match_id) + ".pkl"
            output = open(matches[match_id]['gaia_fn'] , 'wb')
            pickle.dump(gaia_objects_dict, output)
            output.close()


            if index % 1000 == 0:
                #persistate inputs in pkl file
                matches[match_id]['input_fn'] = directory + "inputs/over_time/" + str(match_id) + "_" + str(index) + ".pkl"
                output = open(matches[match_id]['input_fn'] , 'wb')
                pickle.dump(inputs_objects_dict, output)
                output.close()

                #persitate gaia_data
                matches[match_id]['gaia_fn'] = directory + "gaia_data/over_time/" + str(match_id) + "_" + str(index) + ".pkl"
                output = open(matches[match_id]['gaia_fn'] , 'wb')
                pickle.dump(gaia_objects_dict, output)
                output.close()

                #persitate matches
                file_name = directory + "parsed_matches/over_time/" + "_" + str(index) + ".pkl"
                output = open(file_name, 'wb')
                pickle.dump(matches, output)
                output.close()



    except Exception as e:
        #print(match_id, filepath)
        #print(e, "Match_id:", match_id)
        to_be_deleted.append(match_id)
        error_count +=1
        if error_count >= 100:
          pass#break
        pass

    if parse_recordfiles:
        parsed_files += 1
        print("Parsed:" , parsed_files, "- error on: ", error_count,  end='\r')

#errorhandling
print("Parsed:" , parsed_files, "- error on: ", error_count)
print("Deleting: ", to_be_deleted)
for match_id in to_be_deleted:
    del matches[match_id]

if parse_recordfiles:
    #presistate full input dict
    full_fn = input_path + 'all_inputs.pkl'
    output = open(full_fn , 'wb')
    pickle.dump(input_dict, output)
    output.close()

    print("Error on ",error_count," matches while parsing recordfile data")
        
        

In [ ]:
#persitate matches
file_name = 'aoe_data/data/scraped_matches/parsed_matches/final_matches.pkl'
output = open(file_name, 'wb')
pickle.dump(matches, output)
output.close()

In [ ]:
ls

In [ ]:
matches_df = pd.DataFrame.from_dict(matches, orient='index')

matches_df.shape

In [ ]:
ls

In [ ]:
#manual add masterdata
masterdata_dict['Ages'] = ['Feudal Age','Castle Age','Imperial Age']

In [ ]:
for key in masterdata_dict:
    print(key, ": ",len(masterdata_dict[key]))

#persistate master data
masterdata_path = 'aoe_data/masterdata/'

output = open(masterdata_path + 'masterdata_dict.pkl' , 'wb')
pickle.dump(masterdata_dict, output)
output.close()


In [ ]:
matches_df.shape

In [ ]:
matches_df.columns

In [ ]:
#write preprocessed matches_df into csv
matches_df.to_csv('aoe_data/data/scraped_recordfile_matches.csv',index = False)

In [ ]:
matches_df = pd.read_csv('aoe_data/data/scraped_recordfile_matches.csv')
matches_df.head()

In [ ]:
matches_df.groupby(['map']).size().reset_index(name='counts')